# Preparations

In [ ]:
# Do something with Pkg...
import Pkg
Pkg.DEFAULT_IO[] = stdout;

In [ ]:
using BenchmarkTools
using DifferentialEquations
using DiffEqBase
using DiffEqJump
using JSON
using LSODA
using ModelingToolkit
using OrdinaryDiffEq
using Plots; mm = Plots.mm;
using ReactionNetworkImporters
using Sundials

In [ ]:
# Sets plotting defaults
default(framestyle=:box,grid=false,lw=4,la=0.8,guidefontsize=11,legend=:topleft)

In [ ]:
# Lists all possible methods (and jacobian combiantion options).
jac_opts = [(false,false), (true,false), (true,true)]
methods_ode_lsoda_n_CVODE = [(solver=lsoda(),jac_opt=(false,false),name="lsoda (NoLinSol)"), (solver=CVODE_BDF(),jac_opt=(false,false),name="CVODE (NoLinSol)"), (solver=CVODE_BDF(linear_solver=:LapackDense),jac_opt=(false,false),name="CVODE (LapackDense)"), (solver=CVODE_BDF(linear_solver=:GMRES),jac_opt=(false,false),name="CVODE (GMRES)"), (solver=CVODE_BDF(linear_solver=:LapackDense),jac_opt=(true,false),name="CVODE (jac, LapackDense)"), (solver=CVODE_BDF(linear_solver=:KLU),jac_opt=(true,true),name="CVODE (jac, sparse, KLU)")]
methods_ode_other = [(solver=TRBDF2(),jac_opt=(false,false),name="TRBDF2 (NoLinSol)"), (solver=KenCarp4(),jac_opt=(false,false),name="KenCarp4 (NoLinSol)"), (solver=QNDF(),jac_opt=(false,false),name="QNDF (NoLinSol)"), (solver=FBDF(),jac_opt=(false,false),name="FBDF (NoLinSol)"), (solver=Rosenbrock23(),jac_opt=(false,false),name="Rosenbrock23 (NoLinSol)")]
methods_ode_other_jac = [(solver=TRBDF2(),jac_opt=(true,false),name="TRBDF2 (jac, NoLinSol)"), (solver=KenCarp4(),jac_opt=(true,false),name="KenCarp4 (jac, NoLinSol)"), (solver=QNDF(), jac_opt=(true,false),name="QNDF (jac, NoLinSol)"), (solver=FBDF(),jac_opt=(true,false),name="FBDF (jac, NoLinSol)"), (solver=Rosenbrock23(),jac_opt=(true,false),name="Rosenbrock23 (jac, NoLinSol)")]
methods_ode_other_jac_sparse = [(solver=TRBDF2(linsolve=KLUFactorization()),jac_opt=(true,true),name="TRBDF2 (jac, sparse, KLUFactorization)"), (solver=KenCarp4(linsolve=KLUFactorization()),jac_opt=(true,true),name="KenCarp4 (jac, sparse, KLUFactorization)"), (solver=QNDF(linsolve=KLUFactorization()),jac_opt=(true,true),name="QNDF (jac, sparse, KLUFactorization)"), (solver=FBDF(linsolve=KLUFactorization()),jac_opt=(true,true),name="FBDF (jac, sparse, KLUFactorization)"), (solver=Rosenbrock23(linsolve=KLUFactorization()),jac_opt=(true,true),name="Rosenbrock23 (jac, sparse, KLUFactorization)")];
methods_ssa = [(solver=Direct(),name="Direct"), (solver=SortingDirect(),name="SortingDirect"), (solver=RSSA(),name="RSSA"), (solver=RSSACR(),name="RSSACR")];

In [ ]:
# Checks the output plots of a set of ode simulations.
function check_plot_ode_sims(oprobs,ode_solvers,modelvars,fig_name;tend=10.0,print=true)
    if print 
        ode_sols = map(ode_solver -> (println(ode_solver.name,":"); @time solve(remake(oprobs[ode_solver.jac_opt],tspan=(0.0,tend)),ode_solver.solver)), ode_solvers);
    else
        ode_sols = map(ode_solver -> solve(remake(oprobs[ode_solver.jac_opt],tspan=(0.0,tend)),ode_solver.solver), ode_solvers);
    end        
    ode_plot_1 = plot(ode_sols[1],vars=modelvars,xguide="Time (Seconds)",yguide="Concentration (au)",title=ode_solvers[1].name,left_margin=15mm)
    ode_plots = map((sol,name) -> plot(sol,vars=modelvars,xguide="Time (Seconds)",yguide="",yticks=[],title=name,left_margin=3mm), ode_sols[2:end], getfield.(ode_solvers,:name)[2:end]);
    ode_plot = plot([ode_plot_1;ode_plots]...,layout=(1,length(ode_solvers)),size=(length(ode_solvers)*450,325),bottom_margin=13mm,top_margin=4mm)
    savefig(ode_plot,"../Plots/Catalyst/$(fig_name).png")
    savefig(ode_plot,"../Plots/Catalyst/$(fig_name).pdf")
    savefig(ode_plot,"../Plots/Catalyst/$(fig_name).svg")
    return ode_plot
end
# Checks that a set of ode solvers simualte without error.
function check_ode_sims(oprobs,osolvers)
    foreach(ode_solver -> (println(ode_solver.name,":"); @time solve(oprobs[ode_solver.jac_opt],ode_solver.solver); println();), osolvers);
end;

# Checks a set of ssa solvers on a model.
function check_plot_ssa_sims(dprob,ssa_solvers,modelvars,fig_name,rn;tend=10.0)
    j_probs = map(ssa_solver -> JumpProblem(rn,remake(dprob,tspan=(0.0,tend)),ssa_solver.solver,save_positions=(false,false)), ssa_solvers);
    ssa_sols = map((j_prob,name) -> (println(name,":"); @time solve(j_prob,SSAStepper(),saveat=tend/500.0);), j_probs, getfield.(ssa_solvers,:name));
    ssa_plot_1 = plot(ssa_sols[1],vars=modelvars,xguide="Time (Seconds)",yguide="Copy numbers",title=ssa_solvers[1].name,left_margin=15mm)
    ssa_plots = map((sol,name) -> plot(sol,vars=modelvars,xguide="Time (Seconds)",yguide="",yticks=[],title=name,left_margin=3mm), ssa_sols[2:end], getfield.(ssa_solvers,:name)[2:end]);
    ssa_plot = plot([ssa_plot_1;ssa_plots]...,layout=(1,length(ssa_sols)),size=(length(ssa_sols)*450,325),bottom_margin=13mm,top_margin=4mm)
    savefig(ssa_plot,"../Plots/Catalyst/$(fig_name).png")
    savefig(ssa_plot,"../Plots/Catalyst/$(fig_name).pdf")
    savefig(ssa_plot,"../Plots/Catalyst/$(fig_name).svg")
    return ssa_plot
end
# Checks that a set of ssa solvers simualte without error.
function check_ssa_sims(dprob,ssa_solvers,rn)
    j_probs = map(ssa_solver -> JumpProblem(rn,dprob,ssa_solver.solver,save_positions=(false,false)), ssa_solvers);
    foreach((j_prob,name) -> (println(name,":"); @time solve(j_prob,SSAStepper(),saveat=dprob.tspan[end]/500.0); println();), j_probs, getfield.(ssa_solvers,:name));
end;

# Analysis

### Multistate

In [ ]:
# Load model.
multistate_model = loadrxnetwork(BNGNetwork(), "../Data/multistate.net");
@unpack A_P, A_unbound_P, A_bound_P, RLA_P = multistate_model.rn; multistate_obs = [A_P, A_unbound_P, A_bound_P, RLA_P]
multistate_model_no_obs = loadrxnetwork(BNGNetwork(), "../Data/multistate_no_obs.net");

# ODEProblems
multistate_odeprobs = Dict(map(jac_opt -> jac_opt => ODEProblem(convert(ODESystem,multistate_model.rn),Float64[],(0.,10.0),Float64[],jac=jac_opt[1],sparse=jac_opt[2]), jac_opts))
multistate_odeprobs_no_obs = Dict(map(jac_opt -> jac_opt => ODEProblem(convert(ODESystem,multistate_model_no_obs.rn),Float64[],(0.,10.0),Float64[],jac=jac_opt[1],sparse=jac_opt[2]), jac_opts))

# DiscreteProblems
multistate_dprob = DiscreteProblem(multistate_model.rn,multistate_model.u₀,(0.,10.0),multistate_model.p); multistate_dprob = remake(multistate_dprob,u0=Int64.(multistate_dprob.u0));
multistate_dprob_no_obs = DiscreteProblem(multistate_model_no_obs.rn,multistate_model_no_obs.u₀,(0.,10.0),multistate_model_no_obs.p); multistate_dprob_no_obs = remake(multistate_dprob_no_obs,u0=Int64.(multistate_dprob_no_obs.u0));

#### ODE Simulations

In [ ]:
# Checks that the ode output looks correct.
check_plot_ode_sims(multistate_odeprobs,methods_ode_lsoda_n_CVODE,multistate_obs,"multistate_ode_short")

In [ ]:
# Checks that the ode output looks correct over longer simulations.
check_plot_ode_sims(multistate_odeprobs,methods_ode_lsoda_n_CVODE,multistate_obs,"multistate_ode_long";tend=100000.0)

In [ ]:
# Checks for additional solvers (not running for long, just creates way to many simulations...).
display(check_plot_ode_sims(multistate_odeprobs,methods_ode_other,multistate_obs,"OtherSolvers/multistate_ode_other_short";print=false))
display(check_plot_ode_sims(multistate_odeprobs,methods_ode_other_jac,multistate_obs,"OtherSolvers/multistate_ode_other_jac_short";print=false))
display(check_plot_ode_sims(multistate_odeprobs,methods_ode_other_jac_sparse,multistate_obs,"OtherSolvers/multistate_ode_other_jac_sparse_short";print=false))

In [ ]:
# Checks ode simulation for the no_obs models.
check_ode_sims(multistate_odeprobs_no_obs,methods_ode_lsoda_n_CVODE)

#### Jump Simulations

In [ ]:
# Checks that the ssa output looks correct.
check_plot_ssa_sims(multistate_dprob,methods_ssa,multistate_obs,"multistate_ssa_short",multistate_model.rn)

In [ ]:
# Checks that the ssa output looks correct over longer simulations.
check_plot_ssa_sims(multistate_dprob,methods_ssa,multistate_obs,"multistate_ssa_long",multistate_model.rn;tend=100000.0)

In [ ]:
# Checks ssa simulation for the no_obs models.
check_ssa_sims(multistate_dprob_no_obs,methods_ssa,multistate_model.rn)

### Multisite2

In [ ]:
# Load model.
multisite2_model = loadrxnetwork(BNGNetwork(), "../Data/multisite2.net");
@unpack Rfree,Lfree,A1P = multisite2_model.rn; multisite2_obs = [Rfree,Lfree,A1P]
multisite2_model_no_obs = loadrxnetwork(BNGNetwork(), "../Data/multisite2_no_obs.net");

# ODEProblems
multisite2_odeprobs = Dict(map(jac_opt -> jac_opt => ODEProblem(convert(ODESystem,multisite2_model.rn),Float64[],(0.,10.0),Float64[],jac=jac_opt[1],sparse=jac_opt[2]), jac_opts))
multisite2_odeprobs_no_obs = Dict(map(jac_opt -> jac_opt => ODEProblem(convert(ODESystem,multisite2_model_no_obs.rn),Float64[],(0.,10.0),Float64[],jac=jac_opt[1],sparse=jac_opt[2]), jac_opts))

# DiscreteProblems
multisite2_dprob = DiscreteProblem(multisite2_model.rn,multisite2_model.u₀,(0.,10.0),multisite2_model.p); multisite2_dprob = remake(multisite2_dprob,u0=Int64.(multisite2_dprob.u0));
multisite2_dprob_no_obs = DiscreteProblem(multisite2_model_no_obs.rn,multisite2_model_no_obs.u₀,(0.,10.0),multisite2_model_no_obs.p); multisite2_dprob_no_obs = remake(multisite2_dprob_no_obs,u0=Int64.(multisite2_dprob_no_obs.u0));

In [ ]:
# Checks that the ode output looks correct.
check_plot_ode_sims(multisite2_odeprobs,methods_ode_lsoda_n_CVODE,multisite2_obs,"multisite2_ode_short")

In [ ]:
# Checks that the ode output looks correct over longer simulations.
check_plot_ode_sims(multisite2_odeprobs,methods_ode_lsoda_n_CVODE,multisite2_obs,"multisite2_ode_long";tend=10000.0)

In [ ]:
# Checks for additional solvers (not running for long, just creates way to many simulations...).
display(check_plot_ode_sims(multisite2_odeprobs,methods_ode_other,multisite2_obs,"OtherSolvers/multisite2_ode_other_short";print=false))
display(check_plot_ode_sims(multisite2_odeprobs,methods_ode_other_jac,multisite2_obs,"OtherSolvers/multisite2_ode_other_jac_short";print=false))
display(check_plot_ode_sims(multisite2_odeprobs,methods_ode_other_jac_sparse,multisite2_obs,"OtherSolvers/multisite2_ode_other_jac_sparse_short";print=false))

In [ ]:
# Checks ode simulation for the no_obs models.
check_ode_sims(multisite2_odeprobs_no_obs,methods_ode_lsoda_n_CVODE)

#### Jump Simulations

In [ ]:
# Checks that the ssa output looks correct.
check_plot_ssa_sims(multisite2_dprob,methods_ssa,multisite2_obs,"multisite2_ssa_short",multisite2_model.rn)

In [ ]:
# Checks that the ssa output looks correct over longer simulations.
check_plot_ssa_sims(multisite2_dprob,methods_ssa,multisite2_obs,"multisite2_ssa_long",multisite2_model.rn;tend=10000.0)

In [ ]:
# Checks ssa simulation for the no_obs models.
check_ssa_sims(multisite2_dprob_no_obs,methods_ssa,multisite2_model.rn)

### Egfr_net

In [ ]:
# Load model.
egfr_net_model = loadrxnetwork(BNGNetwork(), "../Data/egfr_net.net");
@unpack Dimers,Sos_act,Y1068,Y1148,Shc_Grb,Shc_Grb_Sos,R_Grb2,R_Shc,R_ShcP,ShcP,R_G_S,R_S_G_S,Efgr_tot = egfr_net_model.rn; egfr_net_obs = [Dimers,Sos_act,Y1068,Y1148,Shc_Grb,Shc_Grb_Sos,R_Grb2,R_Shc,R_ShcP,ShcP,R_G_S,R_S_G_S,Efgr_tot]
egfr_net_model_no_obs = loadrxnetwork(BNGNetwork(), "../Data/egfr_net_no_obs.net");

# ODEProblems
egfr_net_odeprobs = Dict(map(jac_opt -> jac_opt => ODEProblem(convert(ODESystem,egfr_net_model.rn),Float64[],(0.,10.0),Float64[],jac=jac_opt[1],sparse=jac_opt[2]), jac_opts))
egfr_net_odeprobs_no_obs = Dict(map(jac_opt -> jac_opt => ODEProblem(convert(ODESystem,egfr_net_model_no_obs.rn),Float64[],(0.,10.0),Float64[],jac=jac_opt[1],sparse=jac_opt[2]), jac_opts))

# DiscreteProblems
egfr_net_dprob = DiscreteProblem(egfr_net_model.rn,egfr_net_model.u₀,(0.,10.0),egfr_net_model.p); egfr_net_dprob = remake(egfr_net_dprob,u0=Int64.(egfr_net_dprob.u0));
egfr_net_dprob_no_obs = DiscreteProblem(egfr_net_model_no_obs.rn,egfr_net_model_no_obs.u₀,(0.,10.0),egfr_net_model_no_obs.p); egfr_net_dprob_no_obs = remake(egfr_net_dprob_no_obs,u0=Int64.(egfr_net_dprob_no_obs.u0));

In [ ]:
# Checks that the ode output looks correct.
check_plot_ode_sims(egfr_net_odeprobs,methods_ode_lsoda_n_CVODE,egfr_net_obs,"egfr_net_ode_short")

In [ ]:
# Checks that the ode output looks correct over longer simulations.
check_plot_ode_sims(egfr_net_odeprobs,methods_ode_lsoda_n_CVODE,egfr_net_obs,"egfr_net_ode_long";tend=1000.0)

In [ ]:
# Checks for additional solvers (not running for long, just creates way to many simulations...).
display(check_plot_ode_sims(egfr_net_odeprobs,methods_ode_other,egfr_net_obs,"OtherSolvers/egfr_net_ode_other_short";print=false))
display(check_plot_ode_sims(egfr_net_odeprobs,methods_ode_other_jac,egfr_net_obs,"OtherSolvers/egfr_net_ode_other_jac_short";print=false))
display(check_plot_ode_sims(egfr_net_odeprobs,methods_ode_other_jac_sparse,egfr_net_obs,"OtherSolvers/egfr_net_ode_other_jac_sparse_short";print=false))

In [ ]:
# Checks ode simulation for the no_obs models.
check_ode_sims(egfr_net_odeprobs_no_obs,methods_ode_lsoda_n_CVODE)

#### Jump Simulations

In [ ]:
# Checks that the ssa output looks correct.
check_plot_ssa_sims(egfr_net_dprob,methods_ssa,egfr_net_obs,"egfr_net_ssa_short",egfr_net_model.rn)

In [ ]:
# Checks that the ssa output looks correct over longer simulations.
check_plot_ssa_sims(egfr_net_dprob,methods_ssa,egfr_net_obs,"egfr_net_ssa_long",egfr_net_model.rn;tend=1000.0)

In [ ]:
# Checks ssa simulation for the no_obs models.
check_ssa_sims(egfr_net_dprob_no_obs,methods_ssa,egfr_net_model.rn)

### BCR

In [ ]:
# Load model.
BCR_model = loadrxnetwork(BNGNetwork(), "../Data/BCR.net");
@unpack Activated_Syk,Ig_alpha_P,Ig_alpha_PP,Ig_beta_PP,Activated_Lyn,Autoinhibited_Lyn,Activated_Fyn,Autoinhibited_Fyn,PAG1_Csk = BCR_model.rn; BCR_obs = [Activated_Syk,Ig_alpha_P,Ig_alpha_PP,Ig_beta_PP,Activated_Lyn,Autoinhibited_Lyn,Activated_Fyn,Autoinhibited_Fyn,PAG1_Csk]
BCR_model_no_obs = loadrxnetwork(BNGNetwork(), "../Data/BCR_no_obs.net");
BCR_ssa_model = loadrxnetwork(BNGNetwork(), "../Data/postequil_c3.net");

# ODEProblems
BCR_odeprobs = Dict(map(jac_opt -> jac_opt => ODEProblem(convert(ODESystem,BCR_model.rn),Float64[],(0.,10.0),Float64[],jac=jac_opt[1],sparse=jac_opt[2]), jac_opts))
BCR_odeprobs_no_obs = Dict(map(jac_opt -> jac_opt => ODEProblem(convert(ODESystem,BCR_model_no_obs.rn),Float64[],(0.,10.0),Float64[],jac=jac_opt[1],sparse=jac_opt[2]), jac_opts))

# DiscreteProblems
BCR_jump_u₀ = JSON.parsefile("../Data/BCR_SSA_u0.json")
BCR_dprob = DiscreteProblem(BCR_ssa_model.rn,BCR_jump_u₀,(0.,10.0),BCR_ssa_model.p); BCR_dprob = remake(BCR_dprob,u0=Int64.(BCR_dprob.u0));

In [ ]:
# Checks that the ode output looks correct.
check_plot_ode_sims(BCR_odeprobs,filter(s->s.jac_opt!=(true,false), methods_ode_lsoda_n_CVODE),BCR_obs,"BCR_ode_short")

In [ ]:
# Checks that the ode output looks correct over longer simulations.
check_plot_ode_sims(BCR_odeprobs,filter(s->s.jac_opt!=(true,false), methods_ode_lsoda_n_CVODE),BCR_obs,"BCR_ode_long";tend=1000.0)

In [ ]:
# Checks for additional solvers (not running for long, just creates way to many simulations...).
#display(check_plot_ode_sims(BCR_odeprobs,methods_ode_other,BCR_obs,"OtherSolvers/BCR_ode_other_short";print=false))             # (Fails due to stack overflow)
#display(check_plot_ode_sims(BCR_odeprobs,methods_ode_other_jac,BCR_obs,"OtherSolvers/BCR_ode_other_jac_short";print=false))     # (Fails due to generation of too large an expression)
display(check_plot_ode_sims(BCR_odeprobs,methods_ode_other_jac_sparse,BCR_obs,"OtherSolvers/BCR_ode_other_jac_sparse_short";print=false))

In [ ]:
# Checks ode simulation for the no_obs models.
check_ode_sims(BCR_odeprobs_no_obs,filter(s->s.jac_opt!=(true,false), methods_ode_lsoda_n_CVODE))

#### Jump Simulations

In [ ]:
# Checks that the ssa output looks correct.
check_plot_ssa_sims(BCR_dprob,methods_ssa[2:end],BCR_obs,"BCR_ssa_short",BCR_ssa_model.rn)

In [ ]:
# Checks that the ssa output looks correct over longer simulations.
check_plot_ssa_sims(BCR_dprob,methods_ssa[2:end],BCR_obs,"BCR_ssa_long",BCR_ssa_model.rn;tend=1000.0)

In [ ]:
# Checks ssa simulation for the no_obs models.
check_ssa_sims(BCR_dprob,methods_ssa[2:end],BCR_ssa_model.rn)

### Fceri_gamma2

In [ ]:
# Load model.
fceri_gamma2_model = loadrxnetwork(BNGNetwork(), "../Data/fceri_gamma2.net");
@unpack LynFree,RecMon,RecPbeta,RecPgamma,RecSyk,RecSykPS = fceri_gamma2_model.rn; fceri_gamma2_obs = [LynFree,RecMon,RecPbeta,RecPgamma,RecSyk,RecSykPS]
fceri_gamma2_model_no_obs = loadrxnetwork(BNGNetwork(), "../Data/fceri_gamma2_no_obs.net");

# ODEProblems
fceri_gamma2_odeprobs = Dict(map(jac_opt -> jac_opt => ODEProblem(convert(ODESystem,fceri_gamma2_model.rn),Float64[],(0.,10.0),Float64[],jac=jac_opt[1],sparse=jac_opt[2]), jac_opts))
fceri_gamma2_odeprobs_no_obs = Dict(map(jac_opt -> jac_opt => ODEProblem(convert(ODESystem,fceri_gamma2_model_no_obs.rn),Float64[],(0.,10.0),Float64[],jac=jac_opt[1],sparse=jac_opt[2]), jac_opts))

# DiscreteProblems
fceri_gamma2_dprob = DiscreteProblem(fceri_gamma2_model.rn,fceri_gamma2_model.u₀,(0.,10.0),fceri_gamma2_model.p); fceri_gamma2_dprob = remake(fceri_gamma2_dprob,u0=Int64.(fceri_gamma2_dprob.u0));
fceri_gamma2_dprob_no_obs = DiscreteProblem(fceri_gamma2_model_no_obs.rn,fceri_gamma2_model_no_obs.u₀,(0.,10.0),fceri_gamma2_model_no_obs.p); fceri_gamma2_dprob_no_obs = remake(fceri_gamma2_dprob_no_obs,u0=Int64.(fceri_gamma2_dprob_no_obs.u0));

In [ ]:
# Checks that the ode output looks correct.
check_plot_ode_sims(fceri_gamma2_odeprobs,filter(s->s.jac_opt!=(true,false), methods_ode_lsoda_n_CVODE),fceri_gamma2_obs,"fceri_gamma2_ode_short")

In [ ]:
# Checks that the ode output looks correct over longer simulations.
check_plot_ode_sims(fceri_gamma2_odeprobs,filter(s->s.jac_opt!=(true,false), methods_ode_lsoda_n_CVODE),fceri_gamma2_obs,"fceri_gamma2_ode_long";tend=1000.0)

In [ ]:
# Checks for additional solvers (not running for long, just creates way to many simulations...).
# display(check_plot_ode_sims(fceri_gamma2_odeprobs,methods_ode_other,fceri_gamma2_obs,"OtherSolvers/fceri_gamma2_ode_other_short";print=false))           # (Fails due to stack overflow)
# display(check_plot_ode_sims(fceri_gamma2_odeprobs,methods_ode_other_jac,fceri_gamma2_obs,"OtherSolvers/fceri_gamma2_ode_other_jac_short";print=false))   # (Fails due to generation of too large an expression)
display(check_plot_ode_sims(fceri_gamma2_odeprobs,methods_ode_other_jac_sparse,fceri_gamma2_obs,"OtherSolvers/fceri_gamma2_ode_other_jac_sparse_short";print=false))

In [ ]:
# Checks ode simulation for the no_obs models.
check_ode_sims(fceri_gamma2_odeprobs_no_obs,filter(s->s.jac_opt!=(true,false), methods_ode_lsoda_n_CVODE))

#### Jump Simulations

In [ ]:
# Checks that the ssa output looks correct.
check_plot_ssa_sims(fceri_gamma2_dprob,methods_ssa,fceri_gamma2_obs,"fceri_gamma2_ssa_short",fceri_gamma2_model.rn)

In [ ]:
# Checks that the ssa output looks correct over longer simulations.
check_plot_ssa_sims(fceri_gamma2_dprob,methods_ssa,fceri_gamma2_obs,"fceri_gamma2_ssa_long",fceri_gamma2_model.rn;tend=1000.0)

In [ ]:
# Checks ssa simulation for the no_obs models.
check_ssa_sims(fceri_gamma2_dprob_no_obs,methods_ssa,fceri_gamma2_model.rn)